In [2]:
#create a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").\
                                     appName("spark_on_docker").\
                                     getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/23 02:56:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
static = spark.read.json("work/TheDefinitiveGuide/Spark-The-Definitive-Guide/data/activity-data/")
dataSchema = static.schema

In [4]:
static.show(3)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|           x|           y|           z|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|   18|nexus4|   g|stand| 3.356934E-4|-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|   66|nexus4|   g|stand|-0.005722046| 0.029083252| 0.005569458|
|1424686735500|1424686733498505625|nexus4_1|   99|nexus4|   g|stand|   0.0078125|-0.017654419| 0.010025024|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
only showing top 3 rows



In [5]:
static.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



Create Streaming DataFrames 

Streaming DataFrames are largely the same as static DataFrames.

In [6]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.json("work/TheDefinitiveGuide/Spark-The-Definitive-Guide/data/activity-data")


Transformations on them to get our data into the correct format. 

Specify transformations on our streaming DataFrame before finally calling an action to start the stream. 

In [7]:
activityCounts = streaming.groupBy("gt").count()

Because this code is being written in local mode on a small machine, we are going to set the
shuffle partitions to a small value to avoid creating too many shuffle partitions:

In [8]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

Need only to specify our action to start the query. 

Specify an output destination, or output sink for ourresult of this query. For this basic example, we are going to write to a memory sink which keeps an in-memory table of the results.

In the process of specifying this sink, we’re going to need to define how Spark will output that data. In this example, we use the complete output mode. This mode rewrites all of the keys along with their counts after every trigger:

We are now writing out our stream! You’ll notice that we set a unique query name to represent
this stream, in this case activity_counts. We specified our format as an in-memory table and
we set the output mode.

In [9]:
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
.format("memory").outputMode("complete")\
.start()

21/12/23 04:33:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d3a26e7d-ec45-453a-80a3-2d4099e23807. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/23 04:33:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
21/12/23 04:33:42 WARN FileStreamSource: Listed 80 file(s) in 5674 ms


After this code is executed, the streaming computation will have started in the background. 

The query object is a handle to that active streaming query, and we must specify that we would like
to wait for the termination of the query using activityQuery.awaitTermination() to prevent
the driver process from exiting while the query is active.

In [ ]:
activityQuery.awaitTermination()

Spark lists this stream, and other active ones, under the active streams in our SparkSession. We
can see a list of those streams by running the following:

In [10]:
spark.streams.active

Spark also assigns each stream a UUID, so if need be you could iterate through the list of
running streams and select the above one. In this case, we assigned it to a variable, so that’s not
necessary.

Now that this stream is running, we can experiment with the results by querying the in-memory
table it is maintaining of the current output of our streaming aggregation. This table will be
called activity_counts, the same as the stream. To see the current data in this output table, we
simply need to query it! We’ll do this in a simple loop that will print the results of the streaming
query every second:

In [ ]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)
